# CP-Divisibility: The Falsification Criterion

**Level**: Advanced  
**Prerequisites**: `spiral_time_intro.ipynb`, `non_markovian_walkthrough.ipynb`

---

## Overview

This notebook explains **CP-divisibility** - the mathematical criterion that allows us to falsify the spiral-time hypothesis.

**Key Point**: From Paper Section 9.2:

> *"The spiral-time memory hypothesis is falsified if, under controlled intervention and reset protocols, process-tensor reconstruction yields a CP-divisible (Markovian) description for all tested multi-time settings."*

Topics covered:
1. What is CP-divisibility?
2. How to test it experimentally
3. Why it's the decisive criterion
4. Implementing CP-divisibility tests

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import expm, logm, sqrtm
from itertools import product
import ipywidgets as widgets
from IPython.display import display, Markdown, Math

np.random.seed(2026)
plt.style.use('seaborn-v0_8-darkgrid')

print("✓ Environment ready")

---

## Part 1: What is CP-Divisibility?

### Dynamical Maps

A **dynamical map** $\mathcal{E}_{t:s}$ describes how a quantum state evolves from time $s$ to time $t$:

$$\rho(t) = \mathcal{E}_{t:s}[\rho(s)]$$

### Complete Positivity (CP)

A map $\mathcal{E}$ is **completely positive** if:
1. It preserves positivity: $\rho \geq 0 \Rightarrow \mathcal{E}[\rho] \geq 0$
2. It remains positive when extended to larger systems: $(\mathcal{E} \otimes \mathcal{I})$ is positive

**Physical meaning**: CP maps are the most general physical evolution of an open quantum system.

### CP-Divisibility (Markovianity)

A process is **CP-divisible** if for all $t \geq s \geq 0$:

$$\mathcal{E}_{t:0} = \mathcal{E}_{t:s} \circ \mathcal{E}_{s:0}$$

where each $\mathcal{E}_{t:s}$ is a completely positive map.

**Physical meaning**: The dynamics can be broken into consecutive CP maps → **memoryless** evolution.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

ax1 = axes[0]
ax1.text(0.1, 0.8, r'$\rho(0)$', fontsize=16, bbox=dict(boxstyle='round', facecolor='lightblue'))
ax1.arrow(0.25, 0.8, 0.15, 0, head_width=0.05, head_length=0.05, fc='blue', ec='blue')
ax1.text(0.35, 0.85, r'$\mathcal{E}_{s:0}$', fontsize=12, color='blue')
ax1.text(0.45, 0.8, r'$\rho(s)$', fontsize=16, bbox=dict(boxstyle='round', facecolor='lightgreen'))
ax1.arrow(0.6, 0.8, 0.15, 0, head_width=0.05, head_length=0.05, fc='green', ec='green')
ax1.text(0.7, 0.85, r'$\mathcal{E}_{t:s}$', fontsize=12, color='green')
ax1.text(0.8, 0.8, r'$\rho(t)$', fontsize=16, bbox=dict(boxstyle='round', facecolor='lightcoral'))
ax1.text(0.1, 0.5, r'$\rho(0)$', fontsize=16, bbox=dict(boxstyle='round', facecolor='lightblue'))
ax1.arrow(0.25, 0.5, 0.5, 0, head_width=0.05, head_length=0.05, fc='purple', ec='purple')
ax1.text(0.4, 0.55, r'$\mathcal{E}_{t:0}$', fontsize=12, color='purple')
ax1.text(0.8, 0.5, r'$\rho(t)$', fontsize=16, bbox=dict(boxstyle='round', facecolor='lightcoral'))
ax1.set_xlim(0, 1)
ax1.set_ylim(0.3, 1)
ax1.axis('off')
ax1.set_title('CP-Divisible (Markovian)', fontsize=14, fontweight='bold')

ax2 = axes[1]
ax2.text(0.1, 0.8, r'$\rho(0)$', fontsize=16, bbox=dict(boxstyle='round', facecolor='lightblue'))
ax2.arrow(0.25, 0.8, 0.5, 0, head_width=0.05, head_length=0.05, fc='red', ec='red', linestyle='--', linewidth=2)
ax2.text(0.4, 0.85, r'$\mathcal{E}_{t:0}$', fontsize=12, color='red')
ax2.text(0.3, 0.65, 'Cannot factorize!', fontsize=11, color='red', style='italic')
ax2.text(0.8, 0.8, r'$\rho(t)$', fontsize=16, bbox=dict(boxstyle='round', facecolor='lightcoral'))
ax2.annotate('', xy=(0.5, 0.75), xytext=(0.6, 0.55), arrowprops=dict(arrowstyle='->', color='orange', lw=2, linestyle=':'))
ax2.text(0.5, 0.5, 'Memory\nfeedback', fontsize=10, color='orange', ha='center')
ax2.set_xlim(0, 1)
ax2.set_ylim(0.3, 1)
ax2.axis('off')
ax2.set_title('Non-CP-Divisible (Non-Markovian)', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

print("\n📐 Key Insight:")
print("  - CP-divisible: Future depends only on present")
print("  - Non-CP-divisible: Future depends on past history")

---

## Part 2: Testing CP-Divisibility

### Method 1: Choi Matrix Criterion

The **Choi matrix** of a map $\mathcal{E}$ is:

$$\Lambda = (\mathcal{I} \otimes \mathcal{E})[|\Phi^+\rangle\langle\Phi^+|]$$

where $|\Phi^+\rangle = \frac{1}{\sqrt{2}}(|00\rangle + |11\rangle)$ is the maximally entangled state.

**Test**: $\mathcal{E}$ is CP if and only if $\Lambda \geq 0$ (positive semidefinite).

In [ ]:
def construct_choi_matrix(kraus_operators):
    d = kraus_operators[0].shape[0]
    choi = np.zeros((d**2, d**2), dtype=complex)
    for K in kraus_operators:
        K_vec = K.flatten()
        choi += np.outer(K_vec, K_vec.conj())
    return choi

def is_completely_positive(choi_matrix, tol=1e-10):
    eigenvalues = np.linalg.eigvalsh(choi_matrix)
    min_eig = np.min(eigenvalues)
    return min_eig > -tol, min_eig

# Example 1: Amplitude damping (CP map)
gamma = 0.3
K0 = np.array([[1, 0], [0, np.sqrt(1-gamma)]], dtype=complex)
K1 = np.array([[0, np.sqrt(gamma)], [0, 0]], dtype=complex)

choi_AD = construct_choi_matrix([K0, K1])
is_cp, min_eig = is_completely_positive(choi_AD)

print("Amplitude Damping:")
print(f"  Is CP: {is_cp}")
print(f"  Min eigenvalue: {min_eig:.6f}")
print("  ✓ Valid CP map")

---

## Part 3: Why CP-Divisibility is the Decisive Criterion

### From Paper Section 9.2 (Criterion 1)

**Falsification criterion**:

> *"The spiral-time memory hypothesis is falsified if, under controlled intervention and reset protocols, process-tensor reconstruction yields a CP-divisible (Markovian) description for all tested multi-time settings within experimental uncertainty."*

### Why this works:

1. **Model-independent**: Doesn't assume specific memory mechanism
2. **Experimentally accessible**: Can be tested via process tomography
3. **Decisive**: Clear yes/no answer
4. **Falsifiable**: Specific prediction that can be proven wrong

### Key Distinction (Section 10.2):

**Environmental non-Markovianity**: CP-divisibility violated with bath  
**Spiral-time memory**: CP-divisibility violated **even in isolation**

---

## Summary

CP-divisibility provides the decisive falsification criterion for spiral-time memory.

### Key Takeaways

1. **CP-divisibility = Markovianity**: A process is Markovian iff it's CP-divisible
2. **Choi matrix test**: Check if intermediate maps have positive Choi matrices
3. **Falsification criterion**: Spiral-time falsified if all processes are CP-divisible

### Next Steps

Continue to:
- `protocol_examples.ipynb`: Implement the three protocols
- `discrimination_workflow.ipynb`: Complete Section 10 workflow

---

## Exercises

In [ ]:
# Exercise 1: Construct Choi matrix for phase damping
# TODO: Phase damping has Kraus operators:
# K0 = [[1, 0], [0, √(1-γ)]]
# K1 = [[0, 0], [0, √γ]]

# Your code here:


# Exercise 2: Test CP-divisibility for power-law kernel
# TODO: Implement evolution with K(Δt) = ε/(1+γΔt)^α

# Your code here:


---

## References

### Paper Sections
- **Section 9**: Process Tensor as Direct Test
- **Section 9.2**: CP-Divisibility Null Hypothesis
- **Section 10.2**: Violation without Environmental Coupling

### Literature
- Choi (1975). "Completely positive linear maps" *Linear Algebra Appl.* **10**, 285
- Pollock et al. (2018). "Non-Markovian quantum processes" *Phys. Rev. Lett.* **120**, 040405
- Milz et al. (2020). "Quantum Stochastic Processes" *PRX Quantum* **2**, 030201